In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [24]:
def doi_url(row):
    query = ''
    if not pd.isna(row['DOI']):
        query = row['DOI']
        return 1,query
    elif not pd.isna(row['Url']):
        query = row['Url']
        return 2,query

In [34]:
df = pd.read_csv('from_zotero.csv') #csv com abstracts faltantes

In [30]:
# Iterate over the DataFrame rows
for index, row in df.iterrows():
    if pd.isna(row['Abstract Note']):
        # Extract the DOI for each row
        op,query = doi_url(row)

        if op == 1:
            doi = query
            article_url = f'https://link.springer.com/{doi}'
        elif op == 2:
            article_url = query

        # Send an HTTP GET request to the article URL
        response = requests.get(article_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the article page
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the abstract section using the HTML structure you provided
            abstract_section = soup.find('section', {'data-title': 'Abstract'})

            if abstract_section:
                # Extract the text content of the abstract
                abstract_text = abstract_section.find('div', {'class': 'c-article-section__content'}).get_text()
                # Update the 'Abstract Note' column in the original DataFrame 'df'
                df.loc[index, 'Abstract Note'] = abstract_text
            else:
                df.loc[index, 'Abstract Note'] = "Abstract not found."
        else:
            df.loc[index, 'Abstract Note'] = "Failed to retrieve data."
            print(f"Failed to retrieve data for DOI {doi}. Status code: {response.status_code}")

In [36]:
df.to_csv('to_rayyan.csv') #novo csv com abstracts preenchidos